In [47]:
import numpy as np
import tqdm
import tensorflow as tf
from nmt_utils import *
import pickle
import random

In [50]:
with open('datasets/dataset','rb') as fb:
    dataset = pickle.load(fb)
    
with open('vocabs/human_vocab','rb') as fb:
    human_vocab = pickle.load(fb)

with open('vocabs/machine_vocab','rb') as fb:
    machine_vocab = pickle.load(fb)

with open('vocabs/inv_machine_vocab','rb') as fb:
    inv_machine_vocab = pickle.load(fb)


m = len(dataset)
random.shuffle(dataset)

# 0.3% test set
train = dataset[:-(m//300)]
test = dataset[-(m//300):]

len(train),len(test)

(10323, 34)

In [51]:
print(human_vocab)

{' ': 0, ',': 1, '.': 2, '/': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, 'a': 14, 'b': 15, 'c': 16, 'd': 17, 'e': 18, 'f': 19, 'i': 20, 'k': 21, 'l': 22, 'm': 23, 'n': 24, 'o': 25, 'p': 26, 'q': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'x': 33, 'y': 34, 'z': 35, 'ç': 36, 'ü': 37, 'ı': 38, 'ş': 39, 'ə': 40, '<unk>': 41, '<pad>': 42}


In [52]:
#maximum length for input and output dates, in order to make all input data in the same length
Tx = 30
Ty = 10  # xxxx-xx-xx output will be in this format so all outputs will be in 10 character long
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

# Each character has its index X and Y we represent data as a list of indices. Then, we change each indices to 
# one hot encoding vector in depth axis 

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (10357, 30)
Y.shape: (10357, 10)
Xoh.shape: (10357, 30, 43)
Yoh.shape: (10357, 10, 11)


In [53]:
def softmax(x, axis=1):
    """Softmax activation function.
    # Arguments
        x : Tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')

In [54]:
len_human_vocab = len(list(human_vocab.keys()))
len_machine_vocab = len(list(machine_vocab.keys()))

n_s = 64 # number of units for the post-attention LSTM's hidden state "s"
n_a = 32 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a' 

In [55]:
from tensorflow.keras import layers

class neural_translation_model(layers.Layer):
    
    def __init__(self, Tx = 30, Ty = 10, n_a = 32, n_s = 64, human_vocab_size = len_human_vocab
                                                           , machine_vocab_size = len_machine_vocab):
        
        # Default parameter for model
        self.Tx = Tx
        self.Ty = Ty        
        self.n_a = n_a # number of units for the pre-attention, bi-directional LSTM's hidden state 'a' 
        self.n_s = n_s # number of units for the post-attention LSTM's hidden state "s"
        self.human_vocab_size = human_vocab_size
        self.machine_vocab_size = machine_vocab_size
        
        
        
        # We will share weights with those layer. In order to prevent them to be intialized for each time step we can either 
        # define them as a global variable or we can create their object
        self.repeator = layers.RepeatVector(Tx)
        self.concatenator =  layers.Concatenate(axis=-1)
        self.densor1 = layers.Dense(10, activation = "tanh")
        self.densor2 = layers.Dense(1, activation = "relu")
        self.activator = layers.Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
        self.dotor = layers.Dot(axes = 1)
        
        self.post_activation_LSTM_cell = layers.LSTM(n_s, return_state = True) # Please do not modify this global variable.
        self.output_layer = layers.Dense(len(machine_vocab), activation=softmax)
        
    def a_step_attention(self, a, s_prev):
        #it is same activation that will be shared for all t_delta activations to calculate alpha
        s_prev = self.repeator(s_prev)
        #concatenate the activations with hidden state of post attention LSTM 
        concatenation = self.concatenator([a,s_prev])
        
        #Here is the small fully connected neural network to find attention weights 
        # intermediate energies
        e = self.densor1(concatenation)
        # Energies
        energies = self.densor2(e)
        #softmax to calculate alphas
        alpha = self.activator(energies)
        
        # context = sum_over_t_x( alpha(t_y,t_x)) * a(t_x)
        context = self.dotor([alpha,a])
        
        return context
    
    def model(self):
        
        X  = layers.Input(shape = (self.Tx,self.human_vocab_size))
        s0 = layers.Input(shape = (self.n_s,), name ='s0')
        c0 = layers.Input(shape = (self.n_s,), name ='c0')
        
        s = s0 
        c = c0 
        
        a = layers.Bidirectional(layers.LSTM(self.n_a ,return_sequences= True))(X)
        
        outputs = []
        
        for t in range(self.Ty):
            
            context = self.a_step_attention(a, s)
            
            s, _, c = self.post_activation_LSTM_cell(context,initial_state=[s, c])
            
            out = self.output_layer(s)
            
            outputs.append(out)
            
        model = tf.keras.Model(inputs = [X,s0,c0] , outputs = outputs)
        
        return model


In [56]:
attention_model = neural_translation_model().model()

In [57]:
attention_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 30, 43)]     0           []                               
                                                                                                  
 s0 (InputLayer)                [(None, 64)]         0           []                               
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 30, 64)      19456       ['input_2[0][0]']                
 )                                                                                                
                                                                                                  
 repeat_vector_1 (RepeatVector)  (None, 30, 64)      0           ['s0[0][0]',               

                                                                  'attention_weights[4][0]',      
                                                                  'bidirectional_1[0][0]',        
                                                                  'attention_weights[5][0]',      
                                                                  'bidirectional_1[0][0]',        
                                                                  'attention_weights[6][0]',      
                                                                  'bidirectional_1[0][0]',        
                                                                  'attention_weights[7][0]',      
                                                                  'bidirectional_1[0][0]',        
                                                                  'attention_weights[8][0]',      
                                                                  'bidirectional_1[0][0]',        
          

In [58]:
opt = tf.keras.optimizers.legacy.Adam(learning_rate = 0.01,beta_1 = 0.9,beta_2 = 0.999,decay = 0.01) 
attention_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

In [59]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [60]:
attention_model.fit([Xoh, s0, c0], outputs, epochs=70, batch_size=16)

Epoch 1/70
648/648 [==============================] - 57s 57ms/step - loss: 8.1927 - dense_5_loss: 0.1972 - dense_5_1_loss: 0.1732 - dense_5_2_loss: 0.9673 - dense_5_3_loss: 1.9011 - dense_5_4_loss: 0.1076 - dense_5_5_loss: 0.3914 - dense_5_6_loss: 1.6577 - dense_5_7_loss: 0.1365 - dense_5_8_loss: 0.9005 - dense_5_9_loss: 1.7603 - dense_5_accuracy: 0.9258 - dense_5_1_accuracy: 0.9522 - dense_5_2_accuracy: 0.6139 - dense_5_3_accuracy: 0.3403 - dense_5_4_accuracy: 0.9848 - dense_5_5_accuracy: 0.8354 - dense_5_6_accuracy: 0.4276 - dense_5_7_accuracy: 0.9737 - dense_5_8_accuracy: 0.6348 - dense_5_9_accuracy: 0.3867
Epoch 2/70
648/648 [==============================] - 35s 53ms/step - loss: 1.6220 - dense_5_loss: 0.0292 - dense_5_1_loss: 0.0170 - dense_5_2_loss: 0.1843 - dense_5_3_loss: 0.2813 - dense_5_4_loss: 0.0018 - dense_5_5_loss: 0.0660 - dense_5_6_loss: 0.4226 - dense_5_7_loss: 0.0054 - dense_5_8_loss: 0.3790 - dense_5_9_loss: 0.2354 - dense_5_accuracy: 0.9888 - dense_5_1_accuracy: 0

648/648 [==============================] - 22s 33ms/step - loss: 0.0574 - dense_5_loss: 9.6447e-04 - dense_5_1_loss: 5.3968e-04 - dense_5_2_loss: 0.0061 - dense_5_3_loss: 0.0233 - dense_5_4_loss: 3.7409e-05 - dense_5_5_loss: 0.0014 - dense_5_6_loss: 0.0066 - dense_5_7_loss: 2.4468e-04 - dense_5_8_loss: 0.0095 - dense_5_9_loss: 0.0087 - dense_5_accuracy: 1.0000 - dense_5_1_accuracy: 1.0000 - dense_5_2_accuracy: 0.9989 - dense_5_3_accuracy: 0.9967 - dense_5_4_accuracy: 1.0000 - dense_5_5_accuracy: 0.9999 - dense_5_6_accuracy: 0.9993 - dense_5_7_accuracy: 1.0000 - dense_5_8_accuracy: 0.9985 - dense_5_9_accuracy: 0.9988
Epoch 15/70
648/648 [==============================] - 22s 34ms/step - loss: 0.0538 - dense_5_loss: 9.6013e-04 - dense_5_1_loss: 5.2808e-04 - dense_5_2_loss: 0.0058 - dense_5_3_loss: 0.0226 - dense_5_4_loss: 3.4906e-05 - dense_5_5_loss: 0.0014 - dense_5_6_loss: 0.0058 - dense_5_7_loss: 2.1823e-04 - dense_5_8_loss: 0.0085 - dense_5_9_loss: 0.0080 - dense_5_accuracy: 0.9999 -

648/648 [==============================] - 22s 33ms/step - loss: 0.0329 - dense_5_loss: 4.1594e-04 - dense_5_1_loss: 2.5535e-04 - dense_5_2_loss: 0.0032 - dense_5_3_loss: 0.0184 - dense_5_4_loss: 1.6483e-05 - dense_5_5_loss: 5.1098e-04 - dense_5_6_loss: 0.0025 - dense_5_7_loss: 8.4468e-05 - dense_5_8_loss: 0.0030 - dense_5_9_loss: 0.0045 - dense_5_accuracy: 1.0000 - dense_5_1_accuracy: 1.0000 - dense_5_2_accuracy: 0.9994 - dense_5_3_accuracy: 0.9967 - dense_5_4_accuracy: 1.0000 - dense_5_5_accuracy: 1.0000 - dense_5_6_accuracy: 0.9998 - dense_5_7_accuracy: 1.0000 - dense_5_8_accuracy: 0.9998 - dense_5_9_accuracy: 0.9993
Epoch 27/70
648/648 [==============================] - 22s 33ms/step - loss: 0.0319 - dense_5_loss: 3.8360e-04 - dense_5_1_loss: 2.4560e-04 - dense_5_2_loss: 0.0030 - dense_5_3_loss: 0.0181 - dense_5_4_loss: 1.5154e-05 - dense_5_5_loss: 4.8732e-04 - dense_5_6_loss: 0.0024 - dense_5_7_loss: 7.7091e-05 - dense_5_8_loss: 0.0028 - dense_5_9_loss: 0.0044 - dense_5_accuracy: 

648/648 [==============================] - 23s 35ms/step - loss: 0.0248 - dense_5_loss: 2.7355e-04 - dense_5_1_loss: 1.8183e-04 - dense_5_2_loss: 0.0020 - dense_5_3_loss: 0.0160 - dense_5_4_loss: 1.0133e-05 - dense_5_5_loss: 3.0210e-04 - dense_5_6_loss: 0.0015 - dense_5_7_loss: 5.2112e-05 - dense_5_8_loss: 0.0016 - dense_5_9_loss: 0.0030 - dense_5_accuracy: 1.0000 - dense_5_1_accuracy: 1.0000 - dense_5_2_accuracy: 0.9995 - dense_5_3_accuracy: 0.9971 - dense_5_4_accuracy: 1.0000 - dense_5_5_accuracy: 1.0000 - dense_5_6_accuracy: 1.0000 - dense_5_7_accuracy: 1.0000 - dense_5_8_accuracy: 0.9999 - dense_5_9_accuracy: 0.9993
Epoch 39/70
648/648 [==============================] - 19s 30ms/step - loss: 0.0245 - dense_5_loss: 2.6858e-04 - dense_5_1_loss: 1.7768e-04 - dense_5_2_loss: 0.0020 - dense_5_3_loss: 0.0158 - dense_5_4_loss: 1.0235e-05 - dense_5_5_loss: 3.2489e-04 - dense_5_6_loss: 0.0014 - dense_5_7_loss: 4.5499e-05 - dense_5_8_loss: 0.0016 - dense_5_9_loss: 0.0029 - dense_5_accuracy: 

648/648 [==============================] - 19s 30ms/step - loss: 0.0208 - dense_5_loss: 2.1162e-04 - dense_5_1_loss: 1.4498e-04 - dense_5_2_loss: 0.0015 - dense_5_3_loss: 0.0145 - dense_5_4_loss: 8.9086e-06 - dense_5_5_loss: 2.3463e-04 - dense_5_6_loss: 0.0011 - dense_5_7_loss: 3.6400e-05 - dense_5_8_loss: 0.0011 - dense_5_9_loss: 0.0021 - dense_5_accuracy: 1.0000 - dense_5_1_accuracy: 1.0000 - dense_5_2_accuracy: 0.9996 - dense_5_3_accuracy: 0.9972 - dense_5_4_accuracy: 1.0000 - dense_5_5_accuracy: 1.0000 - dense_5_6_accuracy: 1.0000 - dense_5_7_accuracy: 1.0000 - dense_5_8_accuracy: 1.0000 - dense_5_9_accuracy: 0.9996
Epoch 51/70
648/648 [==============================] - 19s 30ms/step - loss: 0.0206 - dense_5_loss: 2.1036e-04 - dense_5_1_loss: 1.4251e-04 - dense_5_2_loss: 0.0015 - dense_5_3_loss: 0.0143 - dense_5_4_loss: 8.4842e-06 - dense_5_5_loss: 2.2991e-04 - dense_5_6_loss: 0.0010 - dense_5_7_loss: 3.5864e-05 - dense_5_8_loss: 0.0011 - dense_5_9_loss: 0.0020 - dense_5_accuracy: 

KeyboardInterrupt: 

In [61]:
EXAMPLES = ['10 iyun 2025', '21 avqust 2016', '10 iyun 2007', 'Şənbə May 9 2018', 'Mart 3 2001', '1 mart 2001','aprelin 18-də 98']
s00 = np.zeros((1, n_s))
c00 = np.zeros((1, n_s))
for example in EXAMPLES:
    example = example.lower().replace('-',' ')
    source = string_to_int(example, Tx, human_vocab)
    source = string_to_int(example, Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    source = np.swapaxes(source, 0, 1)
    source = np.expand_dims(source, axis=0)
    prediction = attention_model.predict([source, s00, c00])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    print("source:", example)
    print("output:", ''.join(output),"\n")

1/1 [==============================] - 5s 5s/step
source: 10 iyun 2025
output: 2025-06-10 

1/1 [==============================] - 0s 27ms/step
source: 21 avqust 2016
output: 2016-08-21 

1/1 [==============================] - 0s 28ms/step
source: 10 iyun 2007
output: 2007-06-10 

1/1 [==============================] - 0s 31ms/step
source: şənbə may 9 2018
output: 2018-05-09 

1/1 [==============================] - 0s 31ms/step
source: mart 3 2001
output: 2001-03-03 

1/1 [==============================] - 0s 36ms/step
source: 1 mart 2001
output: 2001-03-01 

1/1 [==============================] - 0s 28ms/step
source: aprelin 18 də 98
output: 1998-04-18 



In [62]:
attention_model.save_weights('models/58epochs_weights.h5')

In [66]:
test_list = [test_sample[0] for test_sample in test]
test_list

['fevral 18 2019',
 '4 oktyabr 1973',
 '7 iyun 2000',
 '23.08.94',
 '17 iyun 2007',
 '28 iyun 1974 cümə',
 'avqustun 17 də 1990',
 '20 fev 1979',
 'mart 15 1998',
 '18 may 1995 cümə axşamı',
 '6 may 1991 bazar ertəsi',
 '17 yanvar 2004 şənbə',
 '13 yan 2012',
 '2 mart 2023 cümə axşamı',
 '13 noy 1999',
 '17.11.09',
 '20 oktyabr 2001 şənbə',
 '28 mart 1973 çərşənbə',
 '19 iyul 2018',
 '22 oktyabr 1994',
 '19 yanvar 2013 şənbə',
 '9 may 1998',
 'oktyabrın 19 da 1972',
 '21 okt 2005',
 '17 noyabr 2007 şənbə',
 '13 avqust 1974 çərşənbə axşamı',
 '15 iyun 1984',
 '20 oktyabr 1995 cümə',
 '29 aprel 1978 şənbə',
 '13 oktyabr 1977 cümə axşamı',
 '27 may 2022 cümə',
 '22 dekabr 2013 bazar',
 '4 iyun 1979',
 '9 iyun 1998 çərşənbə axşamı']

In [67]:
for example in test_list:
    example = example.lower().replace('-',' ')
    source = string_to_int(example, Tx, human_vocab)
    source = string_to_int(example, Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    source = np.swapaxes(source, 0, 1)
    source = np.expand_dims(source, axis=0)
    prediction = attention_model.predict([source, s00, c00])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    print("source:", example)
    print("output:", ''.join(output),"\n")

1/1 [==============================] - 0s 30ms/step
source: fevral 18 2019
output: 2019-02-18 

1/1 [==============================] - 0s 29ms/step
source: 4 oktyabr 1973
output: 1973-10-04 

1/1 [==============================] - 0s 28ms/step
source: 7 iyun 2000
output: 2000-06-07 

1/1 [==============================] - 0s 30ms/step
source: 23.08.94
output: 1994-08-23 

1/1 [==============================] - 0s 31ms/step
source: 17 iyun 2007
output: 2007-06-17 

1/1 [==============================] - 0s 30ms/step
source: 28 iyun 1974 cümə
output: 1974-06-28 

1/1 [==============================] - 0s 32ms/step
source: avqustun 17 də 1990
output: 1990-08-17 

1/1 [==============================] - 0s 30ms/step
source: 20 fev 1979
output: 1979-02-20 

1/1 [==============================] - 0s 29ms/step
source: mart 15 1998
output: 1998-03-15 

1/1 [==============================] - 0s 32ms/step
source: 18 may 1995 cümə axşamı
output: 1995-05-18 

1/1 [==============================] - 